In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,confusion_matrix,roc_auc_score
import classla

In [15]:
train_df=pd.read_csv("train.tsv",sep='\t')
test_df=pd.read_csv("test.tsv",sep='\t')
eval_df=pd.read_csv("eval.tsv",sep='\t')

sve=pd.concat([train_df,test_df,eval_df])
print(f"Total data: {len(sve)}")

Total data: 10388


In [16]:
binary_data=sve[sve['label'].isin([0,2])].copy()
binary_data['sentiment']=binary_data['label'].map({0:0,2:1})
print(f"Binary data: {len(binary_data)}")
print(f"Pozitivni: {len(binary_data[binary_data['sentiment']==1])}")
print(f"Negativni: {len(binary_data[binary_data['sentiment']==0])}")

binary_data.to_csv("Binary data", index=False, encoding='UTF-8')

Binary data: 3321
Pozitivni: 2031
Negativni: 1290


In [18]:
nlp=classla.Pipeline('hr',processors='tokenize,pos,lemma', verbose=False,use_gpu=False)

stop_words = {'i', 'je', 'u', 'na', 'se', 'da', 'su', 'za', 'sa', 's', 'od', 'do', 'po', 'iz', 'o', 
              'a', 'ali', 'ili', 'pa', 'te', 'kad', 'kao', 'što', 'gdje', 'koji', 
              'koja', 'koje', 'bi', 'će', 'ti', 'mu', 'ga', 'ju', 'me', 'mi', 'si', 'to', 'tu', 
              'ta', 'ova', 'ovaj', 'sve', 'svoj', 'ima', 'sam', 'su', 'bio', 'bila', 'bilo'}

In [19]:
def clean_text_content_words_only(text):
    text = str(text).lower()
    doc = nlp(text)
    words = []
    
    # Keep only nouns, verbs, adjectives, and adverbs
    content_pos = {'NOUN', 'VERB', 'ADJ', 'ADV','AUX'}
    
    for sentence in doc.sentences:
        for word in sentence.words:
            if word.upos in content_pos:  # upos = universal part-of-speech tag
                lemma = word.lemma.lower()
                words.append(lemma)
    return ' '.join(words)

In [20]:
import re

def clean_text_content_words_only_neg(text):
    text = str(text).lower()
    text=re.sub(r'[^a-zA-ZčćđšžČĆĐŠŽ\s]','',text)
    text=' '.join(text.split())
    
    # Handle negation before lemmatization - using word boundaries
    text = re.sub(r'\bnije\b', 'NEG_biti', text)
    text = re.sub(r'\bne\b', 'NEG_biti', text)
    text = re.sub(r'\bneću\b', 'NEG_htjeti', text)
    text = re.sub(r'\bnetreba\b', 'NEG_trebati', text)
    text = re.sub(r'\bneće\b', 'NEG_htjeti', text)
    text = re.sub(r'\bnisam\b', 'NEG_biti', text)
    text = re.sub(r'\bnisi\b', 'NEG_biti', text)
    text = re.sub(r'\bnismo\b', 'NEG_biti', text)
    text = re.sub(r'\bniste\b', 'NEG_biti', text)
    text = re.sub(r'\bnisu\b', 'NEG_biti', text)
    text = re.sub(r'\bnećeš\b', 'NEG_htjeti', text)
    text = re.sub(r'\bnećete\b', 'NEG_htjeti', text)
    
    doc = nlp(text)
    words = []
    
    # Keep only nouns, verbs, adjectives, and adverbs
    content_pos = {'NOUN', 'VERB', 'ADJ', 'ADV','AUX'}
    
    for sentence in doc.sentences:
        for word in sentence.words:
            # Handle NEG_ words manually since CLASSLA won't recognize them
            if word.text.startswith('NEG_'):
                words.append(word.text.lower())
            elif word.upos in content_pos:
                lemma = word.lemma.lower()
                words.append(lemma)
    
    return ' '.join(words)

In [21]:
def clean_text_slova(text):
    text=str(text).lower()
    text=re.sub(r'[^a-zA-ZčćđšžČĆĐŠŽ\s]','',text)
    text=' '.join(text.split())
    doc=nlp(text)
    words=[]
    for sentence in doc.sentences:
        for word in sentence.words:
            lemma=word.lemma.lower()
            if len(lemma)>2 and lemma not in stop_words:
                words.append(lemma)
    return ' '.join(words)

In [22]:
def clean_text_slova_neg(text):
    text=str(text).lower()
    text=re.sub(r'[^a-zA-ZčćđšžČĆĐŠŽ\s]','',text)
    text=' '.join(text.split())

    text = re.sub(r'\bnije\b', 'NEG_biti', text)
    text = re.sub(r'\bne\b', 'NEG_biti', text)
    text = re.sub(r'\bneću\b', 'NEG_htjeti', text)
    text = re.sub(r'\bnetreba\b', 'NEG_trebati', text)
    text = re.sub(r'\bneće\b', 'NEG_htjeti', text)
    text = re.sub(r'\bnisam\b', 'NEG_biti', text)
    text = re.sub(r'\bnisi\b', 'NEG_biti', text)
    text = re.sub(r'\bnismo\b', 'NEG_biti', text)
    text = re.sub(r'\bniste\b', 'NEG_biti', text)
    text = re.sub(r'\bnisu\b', 'NEG_biti', text)
    text = re.sub(r'\bnećeš\b', 'NEG_htjeti', text)
    text = re.sub(r'\bnećete\b', 'NEG_htjeti', text)

    doc=nlp(text)
    words=[]
    for sentence in doc.sentences:
        for word in sentence.words:
            if word.text.startswith('NEG_'):
                words.append(word.text.lower())
            else: 
                lemma=word.lemma.lower()
                if (len(lemma)>2 and lemma not in stop_words):
                    words.append(lemma)
    return ' '.join(words)

In [23]:
binary_data_pos=binary_data.copy()
binary_data_pos_neg=binary_data.copy()
binary_data_slova=binary_data.copy()
binary_data_slova_neg=binary_data.copy()
binary_data_slova['processed']=binary_data_slova['text'].apply(clean_text_slova)
binary_data_slova_neg['processed']=binary_data_slova_neg['text'].apply(clean_text_slova_neg)
binary_data_pos['processed']=binary_data_pos['text'].apply(clean_text_content_words_only)
binary_data_pos_neg['processed']=binary_data_pos_neg['text'].apply(clean_text_content_words_only_neg)

In [24]:
def balansiraj_dataset(data):
    negativni=data[data['sentiment']==0]
    pozitivni=data[data['sentiment']==1]
    n_samples=min(len(negativni),len(pozitivni))
    balanced=pd.concat([negativni.sample(n=n_samples,random_state=42),pozitivni.sample(n=n_samples,random_state=42)])
    return balanced

In [25]:
balanced_slova_neg=balansiraj_dataset(binary_data_slova_neg)
balanced_slova=balansiraj_dataset(binary_data_slova)
balanced_pos=balansiraj_dataset(binary_data_pos)
balanced_data_pos_neg=balansiraj_dataset(binary_data_pos_neg)
print(f"SlovaNeg: {len(balanced_slova_neg)}")
print(f"Slova: {len(balanced_slova)}")
print(f"POSNEG: {len(balanced_data_pos_neg)}")
print(f"POS: {len(balanced_pos)}")

SlovaNeg: 2580
Slova: 2580
POSNEG: 2580
POS: 2580


In [26]:
from sklearn.model_selection import cross_val_score, StratifiedKFold

def test_naive_bayes_combined(data, dataset_name):
    print(f"\n{'='*50}")
    print(f"Testiranje: {dataset_name}")
    print(f"{'='*50}")
    X = data['processed']
    y = data['sentiment']

    # Train/Test Split Analysis
    print("TRAIN/TEST SPLIT RESULTS:")
    print("-" * 30)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    print(f"Train uzoraka: {len(X_train)}")
    print(f"Test uzoraka: {len(X_test)}")

    vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1,1))
    X_train_tfidf = vectorizer.fit_transform(X_train)
    X_test_tfidf = vectorizer.transform(X_test)

    model = MultinomialNB(alpha=0.5)
    model.fit(X_train_tfidf, y_train)

    y_pred = model.predict(X_test_tfidf)
    y_proba = model.predict_proba(X_test_tfidf)[:,1]

    accuracy = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_proba)
    cm = confusion_matrix(y_test, y_pred)

    print(f"Accuracy: {accuracy: .3f} ({accuracy:.1%})")
    print(f"AUC: {auc:.3f}")
    print(f"Confusion Matrix:")
    print(cm)
    
    # Save misclassified examples to CSV files
    X_test_original = data.loc[X_test.index, 'text']  # Get original texts
    
    misclassified_data = []
    for i in range(len(y_test)):
        if y_test.iloc[i] != y_pred[i]:
            text = X_test_original.iloc[i]
            actual = y_test.iloc[i]
            predicted = y_pred[i]
            text_length = len(text)
            
            misclassified_data.append({
                'text': text,
                'actual_label': actual,
                'predicted_label': predicted,
                'text_length': text_length,
                'error_type': 'False Positive' if (actual == 0 and predicted == 1) else 'False Negative'
            })
    
    # Convert to DataFrame and sort by length (shortest first)
    misclassified_df = pd.DataFrame(misclassified_data)
    misclassified_df = misclassified_df.sort_values('text_length')
    
    # Save CSV files
    filename_base = dataset_name.replace(' ', '_').lower()
    misclassified_df.to_csv(f'{filename_base}_misclassified.csv', index=False, encoding='utf-8')
    
    false_positives = misclassified_df[misclassified_df['error_type'] == 'False Positive'].sort_values('text_length')
    false_negatives = misclassified_df[misclassified_df['error_type'] == 'False Negative'].sort_values('text_length')
    
    false_positives.to_csv(f'{filename_base}_false_positives.csv', index=False, encoding='utf-8')
    false_negatives.to_csv(f'{filename_base}_false_negatives.csv', index=False, encoding='utf-8')
    
    print(f"\nMisclassified: {len(misclassified_df)} total")
    print(f"False Positives: {len(false_positives)}")
    print(f"False Negatives: {len(false_negatives)}")
    print(f"CSV files saved: {filename_base}_misclassified.csv, {filename_base}_false_positives.csv, {filename_base}_false_negatives.csv")
    
    # Cross-Validation Analysis
    print(f"\n{'='*50}")
    print("CROSS-VALIDATION RESULTS:")
    print("-" * 30)
    
    # TF-IDF vektorizacija na cijelom datasetu
    vectorizer_cv = TfidfVectorizer(max_features=5000, ngram_range=(1,1))
    X_tfidf = vectorizer_cv.fit_transform(X)
    
    # Model
    model_cv = MultinomialNB(alpha=0.5)
    
    # 5-fold stratified CV
    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    
    # CV scores
    cv_scores = cross_val_score(model_cv, X_tfidf, y, cv=cv, scoring='accuracy')
    cv_auc_scores = cross_val_score(model_cv, X_tfidf, y, cv=cv, scoring='roc_auc')
    
    print(f"CV Accuracy scores: {cv_scores}")
    print(f"CV Accuracy: {cv_scores.mean():.3f} ± {cv_scores.std():.3f}")
    print(f"CV AUC scores: {cv_auc_scores}")
    print(f"CV AUC: {cv_auc_scores.mean():.3f} ± {cv_auc_scores.std():.3f}")
    
    return accuracy, auc, cm, cv_scores.mean(), cv_auc_scores.mean()

In [27]:
results={}

In [28]:
results['slova']=test_naive_bayes_combined(balanced_slova,"Slova preprocessing")


Testiranje: Slova preprocessing
TRAIN/TEST SPLIT RESULTS:
------------------------------
Train uzoraka: 2064
Test uzoraka: 516
Accuracy:  0.779 (77.9%)
AUC: 0.858
Confusion Matrix:
[[192  66]
 [ 48 210]]

Misclassified: 114 total
False Positives: 66
False Negatives: 48
CSV files saved: slova_preprocessing_misclassified.csv, slova_preprocessing_false_positives.csv, slova_preprocessing_false_negatives.csv

CROSS-VALIDATION RESULTS:
------------------------------
CV Accuracy scores: [0.81782946 0.82170543 0.7751938  0.77131783 0.80232558 0.79844961
 0.79844961 0.75581395 0.77131783 0.81007752]
CV Accuracy: 0.792 ± 0.021
CV AUC scores: [0.88696593 0.91208461 0.87320474 0.87050057 0.88774713 0.87825251
 0.88864852 0.84937804 0.8453218  0.87723094]
CV AUC: 0.877 ± 0.018


In [29]:
results['slova_neg']=test_naive_bayes_combined(balanced_slova_neg,"Slova_neg preprocessing")


Testiranje: Slova_neg preprocessing
TRAIN/TEST SPLIT RESULTS:
------------------------------
Train uzoraka: 2064
Test uzoraka: 516
Accuracy:  0.773 (77.3%)
AUC: 0.860
Confusion Matrix:
[[191  67]
 [ 50 208]]

Misclassified: 117 total
False Positives: 67
False Negatives: 50
CSV files saved: slova_neg_preprocessing_misclassified.csv, slova_neg_preprocessing_false_positives.csv, slova_neg_preprocessing_false_negatives.csv

CROSS-VALIDATION RESULTS:
------------------------------
CV Accuracy scores: [0.82170543 0.84108527 0.78682171 0.79069767 0.79844961 0.80620155
 0.78294574 0.76744186 0.7751938  0.82170543]
CV Accuracy: 0.799 ± 0.022
CV AUC scores: [0.89009074 0.91496905 0.87638964 0.88095667 0.892855   0.88149751
 0.89027102 0.85460609 0.84862689 0.88438195]
CV AUC: 0.881 ± 0.018


In [30]:
results['pos']=test_naive_bayes_combined(balanced_pos,"POS preprocessing")


Testiranje: POS preprocessing
TRAIN/TEST SPLIT RESULTS:
------------------------------
Train uzoraka: 2064
Test uzoraka: 516
Accuracy:  0.773 (77.3%)
AUC: 0.859
Confusion Matrix:
[[184  74]
 [ 43 215]]

Misclassified: 117 total
False Positives: 74
False Negatives: 43
CSV files saved: pos_preprocessing_misclassified.csv, pos_preprocessing_false_positives.csv, pos_preprocessing_false_negatives.csv

CROSS-VALIDATION RESULTS:
------------------------------
CV Accuracy scores: [0.80620155 0.82945736 0.7751938  0.77906977 0.78682171 0.78294574
 0.81007752 0.75193798 0.79069767 0.77906977]
CV Accuracy: 0.789 ± 0.020
CV AUC scores: [0.87807223 0.91247521 0.85637882 0.86875789 0.8791539  0.87554834
 0.89537888 0.84985878 0.85157142 0.86401058]
CV AUC: 0.873 ± 0.019


In [31]:
results['POSNEG']=test_naive_bayes_combined(balanced_data_pos_neg,"POS preprocessing NEG")


Testiranje: POS preprocessing NEG
TRAIN/TEST SPLIT RESULTS:
------------------------------
Train uzoraka: 2064
Test uzoraka: 516
Accuracy:  0.779 (77.9%)
AUC: 0.863
Confusion Matrix:
[[185  73]
 [ 41 217]]

Misclassified: 114 total
False Positives: 73
False Negatives: 41
CSV files saved: pos_preprocessing_neg_misclassified.csv, pos_preprocessing_neg_false_positives.csv, pos_preprocessing_neg_false_negatives.csv

CROSS-VALIDATION RESULTS:
------------------------------
CV Accuracy scores: [0.80620155 0.82945736 0.7751938  0.78682171 0.79844961 0.78682171
 0.80232558 0.75581395 0.79844961 0.79844961]
CV Accuracy: 0.794 ± 0.019
CV AUC scores: [0.87807223 0.9170122  0.86413076 0.87945436 0.88275945 0.88143741
 0.8944775  0.85472628 0.85316387 0.87326483]
CV AUC: 0.878 ± 0.018


In [32]:
binary_data_6=binary_data_slova[binary_data_slova['processed'].str.len()>6].copy()
balanced_6=balansiraj_dataset(binary_data_6)
print(len(balanced_6))

2548


In [33]:
results['slova6']=test_naive_bayes_combined(balanced_6,"6slova")


Testiranje: 6slova
TRAIN/TEST SPLIT RESULTS:
------------------------------
Train uzoraka: 2038
Test uzoraka: 510
Accuracy:  0.804 (80.4%)
AUC: 0.887
Confusion Matrix:
[[196  59]
 [ 41 214]]

Misclassified: 100 total
False Positives: 59
False Negatives: 41
CSV files saved: 6slova_misclassified.csv, 6slova_false_positives.csv, 6slova_false_negatives.csv

CROSS-VALIDATION RESULTS:
------------------------------
CV Accuracy scores: [0.81960784 0.79607843 0.83921569 0.80392157 0.81960784 0.76862745
 0.79215686 0.76078431 0.83070866 0.78346457]
CV Accuracy: 0.801 ± 0.025
CV AUC scores: [0.90840305 0.88152067 0.89862205 0.87512303 0.9015748  0.87718381
 0.88447343 0.84387303 0.88610577 0.87358175]
CV AUC: 0.883 ± 0.017


In [34]:
binary_data_60=binary_data_slova_neg[binary_data_slova_neg['processed'].str.len()>6].copy()
balanced_60=balansiraj_dataset(binary_data_60)

In [35]:
results['slova_neg6']=test_naive_bayes_combined(balanced_60,"6slova_neg")


Testiranje: 6slova_neg
TRAIN/TEST SPLIT RESULTS:
------------------------------
Train uzoraka: 2044
Test uzoraka: 512
Accuracy:  0.820 (82.0%)
AUC: 0.895
Confusion Matrix:
[[212  44]
 [ 48 208]]

Misclassified: 92 total
False Positives: 44
False Negatives: 48
CSV files saved: 6slova_neg_misclassified.csv, 6slova_neg_false_positives.csv, 6slova_neg_false_negatives.csv

CROSS-VALIDATION RESULTS:
------------------------------
CV Accuracy scores: [0.76953125 0.8359375  0.8359375  0.81640625 0.77734375 0.78515625
 0.78823529 0.82352941 0.81176471 0.81176471]
CV Accuracy: 0.806 ± 0.023
CV AUC scores: [0.85540771 0.909729   0.9085083  0.89007568 0.89611816 0.85217285
 0.87832185 0.900406   0.90877215 0.88127461]
CV AUC: 0.888 ± 0.020


In [36]:
binary_data_20=binary_data_pos_neg[binary_data_pos_neg['processed'].str.len()>6].copy()
balanced_20=balansiraj_dataset(binary_data_20)

In [37]:
results['6pos_neg']=test_naive_bayes_combined(balanced_20,"6pos_neg")


Testiranje: 6pos_neg
TRAIN/TEST SPLIT RESULTS:
------------------------------
Train uzoraka: 2036
Test uzoraka: 510
Accuracy:  0.788 (78.8%)
AUC: 0.879
Confusion Matrix:
[[195  60]
 [ 48 207]]

Misclassified: 108 total
False Positives: 60
False Negatives: 48
CSV files saved: 6pos_neg_misclassified.csv, 6pos_neg_false_positives.csv, 6pos_neg_false_negatives.csv

CROSS-VALIDATION RESULTS:
------------------------------
CV Accuracy scores: [0.79607843 0.80784314 0.80784314 0.79607843 0.81176471 0.81568627
 0.74015748 0.75590551 0.78740157 0.7992126 ]
CV Accuracy: 0.792 ± 0.024
CV AUC scores: [0.87629183 0.88730315 0.88521161 0.88225886 0.90071358 0.86411171
 0.87730175 0.84425569 0.86124372 0.88746977]
CV AUC: 0.877 ± 0.015


In [38]:
binary_data_20=binary_data_pos[binary_data_pos['processed'].str.len()>6].copy()
balanced_20=balansiraj_dataset(binary_data_20)

In [39]:
results['6pos']=test_naive_bayes_combined(balanced_20,"6pos")


Testiranje: 6pos
TRAIN/TEST SPLIT RESULTS:
------------------------------
Train uzoraka: 2028
Test uzoraka: 508
Accuracy:  0.756 (75.6%)
AUC: 0.855
Confusion Matrix:
[[183  71]
 [ 53 201]]

Misclassified: 124 total
False Positives: 71
False Negatives: 53
CSV files saved: 6pos_misclassified.csv, 6pos_false_positives.csv, 6pos_false_negatives.csv

CROSS-VALIDATION RESULTS:
------------------------------
CV Accuracy scores: [0.7519685  0.79527559 0.82677165 0.77559055 0.78740157 0.78346457
 0.74703557 0.7312253  0.80237154 0.7944664 ]
CV Accuracy: 0.780 ± 0.027
CV AUC scores: [0.8512617  0.88238576 0.89673879 0.8520057  0.85963172 0.87866576
 0.86389201 0.84376953 0.87139108 0.88670166]
CV AUC: 0.869 ± 0.016


In [40]:
binary_data_balanced=balansiraj_dataset(binary_data)
binary_data_balanced['processed']=binary_data_balanced['text']
results['original']=test_naive_bayes_combined(binary_data_balanced,"original")


Testiranje: original
TRAIN/TEST SPLIT RESULTS:
------------------------------
Train uzoraka: 2064
Test uzoraka: 516
Accuracy:  0.742 (74.2%)
AUC: 0.817
Confusion Matrix:
[[193  65]
 [ 68 190]]

Misclassified: 133 total
False Positives: 65
False Negatives: 68
CSV files saved: original_misclassified.csv, original_false_positives.csv, original_false_negatives.csv

CROSS-VALIDATION RESULTS:
------------------------------
CV Accuracy scores: [0.80620155 0.77906977 0.79457364 0.80620155 0.71705426 0.76744186
 0.77906977 0.72868217 0.69379845 0.78294574]
CV Accuracy: 0.766 ± 0.037
CV AUC scores: [0.88143741 0.86989965 0.87470705 0.86124632 0.83144042 0.85187188
 0.87230335 0.83423472 0.81052821 0.86070549]
CV AUC: 0.855 ± 0.022
